In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Bibliotecas carregadas!")

In [ ]:
# Carregar dados processados
df = pd.read_csv('../../data/processed/vendas_tratadas.csv')
df['data_venda'] = pd.to_datetime(df['data_venda'])

print(f"📊 {len(df)} vendas carregadas!")
print(f"🗓️  Período: {df['data_venda'].min().date()} até {df['data_venda'].max().date()}")

In [ ]:
# ANÁLISE 1: Top 10 produtos por QUANTIDADE vendida
print("="*70)
print("🏆 TOP 10 PRODUTOS - QUANTIDADE VENDIDA")
print("="*70)

top_qtd = df.groupby('produto')['quantidade'].sum().sort_values(ascending=False).head(10)
print(top_qtd)

In [ ]:
# ANÁLISE 2: Top 10 produtos por RECEITA
print("="*70)
print("💰 TOP 10 PRODUTOS - RECEITA TOTAL")
print("="*70)

top_receita = df.groupby('produto')['valor_total'].sum().sort_values(ascending=False).head(10)
print(top_receita.apply(lambda x: f'R$ {x:,.2f}'))

In [ ]:
# ANÁLISE 3: Análise por CATEGORIA
print("="*70)
print("📦 ANÁLISE POR CATEGORIA")
print("="*70)

cat_analise = df.groupby('categoria').agg({
    'quantidade': 'sum',
    'valor_total': 'sum',
    'id_produto': 'count'
}).rename(columns={'id_produto': 'num_vendas'})

cat_analise['ticket_medio'] = cat_analise['valor_total'] / cat_analise['num_vendas']
cat_analise = cat_analise.sort_values('valor_total', ascending=False)

print(cat_analise)

In [ ]:
# Criar dashboard de produtos
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📊 ANÁLISE DE PRODUTOS E CATEGORIAS', fontsize=18, fontweight='bold', y=1.00)

# Gráfico 1: Top 10 Produtos - Quantidade
top_qtd.sort_values().plot(kind='barh', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_title('Top 10 Produtos - Quantidade Vendida', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Quantidade Total')
axes[0, 0].grid(axis='x', alpha=0.3)

# Gráfico 2: Top 10 Produtos - Receita
top_receita.sort_values().plot(kind='barh', ax=axes[0, 1], color='coral')
axes[0, 1].set_title('Top 10 Produtos - Receita Total (R$)', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Receita (R$)')
axes[0, 1].grid(axis='x', alpha=0.3)
# Formatar valores no eixo X
axes[0, 1].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R$ {x/1000:.0f}K'))

# Gráfico 3: Receita por Categoria
cat_analise['valor_total'].plot(kind='bar', ax=axes[1, 0], color='teal')
axes[1, 0].set_title('Receita Total por Categoria', fontsize=13, fontweight='bold')
axes[1, 0].set_ylabel('Receita (R$)')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R$ {x/1000:.0f}K'))

# Gráfico 4: Distribuição de vendas por categoria (Pizza)
cat_vendas = df['categoria'].value_counts()
colors = sns.color_palette('Set3', len(cat_vendas))
axes[1, 1].pie(cat_vendas, labels=cat_vendas.index, autopct='%1.1f%%', 
               startangle=90, colors=colors)
axes[1, 1].set_title('Distribuição de Vendas por Categoria', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig('../../images/ranking_produtos.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráficos salvos em: images/ranking_produtos.png")

In [ ]:
# Gerar insights automáticos
print("\n" + "="*70)
print("💡 INSIGHTS PRINCIPAIS")
print("="*70)

# Produto campeão
prod_campeao = top_receita.index[0]
receita_campeao = top_receita.values[0]
print(f"\n🥇 Produto com maior receita:")
print(f"   → {prod_campeao}: R$ {receita_campeao:,.2f}")

# Categoria dominante
cat_dominante = cat_analise.index[0]
receita_cat = cat_analise.loc[cat_dominante, 'valor_total']
perc_cat = (receita_cat / df['valor_total'].sum()) * 100
print(f"\n📦 Categoria mais lucrativa:")
print(f"   → {cat_dominante}: R$ {receita_cat:,.2f} ({perc_cat:.1f}% do total)")

# Ticket médio por categoria
cat_maior_ticket = cat_analise['ticket_medio'].idxmax()
valor_ticket = cat_analise.loc[cat_maior_ticket, 'ticket_medio']
print(f"\n💰 Categoria com maior ticket médio:")
print(f"   → {cat_maior_ticket}: R$ {valor_ticket:,.2f}")

# Concentração (top 3 produtos)
top3_receita = top_receita.head(3).sum()
concentracao = (top3_receita / df['valor_total'].sum()) * 100
print(f"\n📊 Concentração de vendas:")
print(f"   → Top 3 produtos representam {concentracao:.1f}% da receita total")

print("\n" + "="*70)